This notebook is intended to be downloaded and run locally, or run in cloud environments with persistent environments, like Sagemaker Studio Lab:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/InsightSoftwareConsortium/itkwidgets/HEAD?labpath=examples%2FEnvironmentCheck.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/InsightSoftwareConsortium/itkwidgets/blob/main/examples/EnvironmentCheck.ipynb)

# Environment Check

#### This notebook is designed to check the environment that you are running in to make sure that all example notebook dependencies and extensions are correctly installed. Simply select Run All Cells and let everything complete before running the example notebooks in this repository.

In [ ]:
import os, sys, re
import importlib.util
try:
    import importlib.metadata as importlib_metadata
except:
    import importlib_metadata

#### Define the function to do the checking and install any missing dependencies

In [ ]:
def _get_version(pkg):
    try:
        return importlib_metadata.version(pkg)
    except:
        pass
    try:
        return sys.modules[pkg].__version__
    except:
        return ''

def _pkg_version_pre(values):
    version, pre = None, False
    if len(values) == 3:
        version, pre = values[1:]
    elif len(values) == 2:
        pre = (values[1] == "pre")
        version = values[1] if not pre else version
    pkg = values[0]

    return pkg, version, pre

def check_for_package(req):
    values = list(filter(None, re.split(r"\[.*\]|==|>=|--| ", req))) # Grab the package name, version, and pre-release status
    install_req = re.split(r" --pre", req)[0] # Grab the string we need for installation
    pkg, version, pre = _pkg_version_pre(values)
    if (importlib.util.find_spec(pkg.replace("-", "_")) is None
        or (version and _get_version(pkg) != version)):
        print(f"{install_req} not found, installing {pkg} now...")
        try:
            if pre:
                !{sys.executable} -m pip install --upgrade --pre -q "{install_req}"
            else:
                !{sys.executable} -m pip install --upgrade -q "{install_req}"
        except Exception as e:
            print(f'ERROR: {e}')
    print(f"{pkg} version {_get_version(pkg)} installed.")
    print("-----")

#### List of notebook requirements

In [ ]:
reqs = [
    "itkwidgets[all]>=1.0a23",
    "imjoy-elfinder",
    "imjoy-jupyter-extension",
    "imjoy-jupyterlab-extension",
    "itk==5.3.0",
    "itk-io==5.3.0",
    "itk-core",
    "monai-weekly[nibabel, matplotlib, tqdm]",
    "imageio",
    "pyvista",
    "dask[diagnostics]",
    "toolz",
    "scikit-image",
    "pooch",
    "matplotlib",
    "tqdm",
    "vtk",
    "netCDF4",
    "xarray",
    "zarr",
    "fsspec[http]",
]

#### Upgrade pip, just in case.

In [ ]:
!{sys.executable} -m pip install --upgrade -q pip

#### Make sure that the package is installed and that it is the correct version.

**WARNING**: Pip will sometimes raise errors for dependency conflicts. This errors can typically be safely ignored, but often times these issues can be avoided all together by creating a new, clean [python virtual environment](https://docs.python.org/3/library/venv.html) or [conda environment](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-environments). You can follow the [Getting Started](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#) instructions if you are setting up conda for the fist time. If you continue to see errors or are unable to run the notebooks in this repo after running this notebook you can also [open an issue](https://github.com/InsightSoftwareConsortium/itkwidgets/issues/new).

In [ ]:
for req in reqs:
    check_for_package(req)

In [ ]:
if os.environ.get('CONDA_DEFAULT_ENV', None):
    !conda install --yes -q --prefix {sys.prefix} -c conda-forge pyimagej
else:
    raise RuntimeError("No conda environment is activated, currently unable to install pyimagej. Please activate a conda environment and re-run this cell.")

#### Special case specific to running in AWS StudioLab

In [ ]:
if "studio-lab-user" in os.getcwd():
    # Make sure that the imjoy extension is installed in the Jupyter environment
    # and not just the kernel environment since they may not be the same
    !conda env update -n studiolab -f ../environment.yml
    !conda install --yes -q --prefix {sys.prefix} -c conda-forge opencv nodejs

#### Make sure that the required extension(s) are loaded.

In [ ]:
%%javascript
let needReload = (typeof window.loadImJoyRPC === "undefined");
if (needReload) {
    needReload = false;
    location.reload();
}